In [1]:
import torch
from torch import nn

In [ ]:
#=nb

In [2]:
img = torch.rand(3,224,224)

In [3]:
class Inception(nn.Module):
  def __init__(self, in_ch, out_ch, classes):
    super().__init__()
    self.conv1 = nn.Conv2d(in_ch, out_ch, kernel_size = 1)
    self.conv2 = nn.Sequential(
        nn.Conv2d(in_ch, out_ch, kernel_size = 1),
        nn.ReLU(),
        nn.Conv2d(out_ch, out_ch, kernel_size = 3, padding = 1),
        nn.ReLU()
    )
    self.conv3 = nn.Sequential(
        nn.Conv2d(in_ch , out_ch, kernel_size = 1),
        nn.ReLU(),
        nn.Conv2d(out_ch, out_ch, kernel_size = 5, padding = 2),
        nn.ReLU()
    )
    self.conv4 = nn.Sequential(
        nn.MaxPool2d(kernel_size = 3, stride = 1, padding = 1, ceil_mode = True),
        nn.ReLU(),
        nn.Conv2d(in_ch, out_ch, kernel_size = 1),
        nn.ReLU()
    )
  def _forward(self, X):
    out1 = self.conv1(X)
    out2 = self.conv2(X)
    out3 = self.conv3(X)
    out4 = self.conv4(X)
    return [out1, out2, out3, out4]
  def forward(self, X):
      output = self._forward(X)
      return torch.cat(output, 1)


In [4]:
class googleLnet(nn.Module):
  def __init__(self, classes):
    super().__init__()
    self.flatten = nn.Flatten()
    self.Conv1 = nn.Sequential(
        nn.Conv2d(in_channel = 3, out_channel = 64, kernel_size = 7, padding = 2, stride = 2),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1),
        nn.ReLU(),
        nn.Conv2d(64, 192, kernel_size = 3, padding = 1, stride = 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1),
        nn.ReLU(),
    )
    self.inception_3a = Inception(192, 256)
    self.inception_3b = Inception(256, 480)
    self.Max1 = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
    self.max = nn.ReLU()
    self.inception_4a = Inception(480, 512)
    self.inception_4b = Inception(512, 512)
    self.inception_4c = Inception(512, 512)
    self.inception_4d = Inception(512, 528)
    self.inception_4e = Inception(528, 832)
    self.Max2 = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
    self.inception_5a = Inception(832, 832)
    self.inception_5b = Inception(832, 1024)
    self.avgPool = nn.AvgPool2d(kernel_size = 7, stride = 1)
    self.fc = nn.Sequential(
        nn.Dropout(0.4),
        nn.Linear(832, classes)
    )
  def forward(self, X):
      X = self.Conv1(1)
      X = self.inception_3a(X)
      X = self.inception_3b(X)
      X = self.Max1(X)
      X = self.max(X)
      X = self.inception_4a(X)
      X = self.inception_4b(X)
      X = self.inception_4c(X)
      X = self.inception_4d(X)
      X = self.inception_4e(X)
      X = self.Max2(X)
      X = self.inception_5a(X)
      X = self.inception_5b(X)
      X = self.acgPool(X)
      X = self.fc(X)
      return X

